# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

**Шаги для выбора локации:**

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.


**Описание данных:**

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

**Условия задачи:**

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

In [31]:
import pandas as pd

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import seaborn as sns
import numpy as np
from scipy import stats as st


In [32]:
d0 = pd.read_csv('/datasets/geo_data_0.csv')
d1 = pd.read_csv('/datasets/geo_data_1.csv')
d2 = pd.read_csv('/datasets/geo_data_2.csv')

In [33]:
df = [d0, d1, d2]
for i in df:
    print(i.info())
    print()
    print('Основные характеристики')
    print(i.describe())
    print()
    print("Пропуски")
    print(i.isna().sum())
    print('---'*25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Основные характеристики
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max     

По предварительной оценке в датасетах отсутствуют пропуски, отличия средних значений и медиан незначительны, следовательно датасеты без выбросов. Проверим датасеты на яные дубликаты. 

In [34]:
for i in df:
    print(i.duplicated().sum())
    print()

0

0

0



Явные дубликаты отсутсвуют, посмотрим на повторяющиеся ID

In [35]:
for i in df:
    # отображение первых повторяющихся строк
    print(i[i.duplicated(['id'],keep='last' )])  
    print()
    # отображение последних повторяющихся строк
    print(i[i.duplicated(['id'],keep='first')])
    print('---'*20)
    

          id        f0        f1         f2     product
931    HZww2  0.755284  0.368511   1.863211   30.681774
1364   bxg6G  0.411645  0.856830  -3.653440   73.604260
1949   QcMuo  0.506563 -0.323775  -2.215583   75.496502
3389   A5aEY -0.039949  0.156872   0.209861   89.249364
16633  fiKDv  0.157341  1.028359   5.585586   95.817889
21426  Tdehs  0.829407  0.298807  -0.049563   96.035308
42529  AGS9W  1.454747 -0.479651   0.683380  126.370504
60140  TtcGQ  0.569276 -0.104876   6.440215   85.350186
64022  74z30  0.741456  0.459229   5.153109  140.771492
89582  bsk9y  0.398908 -0.400253  10.122376  163.433078

          id        f0        f1         f2     product
7530   HZww2  1.061194 -0.373969  10.430210  158.828695
41724  bxg6G -0.823752  0.546319   3.630479   93.007798
51970  A5aEY -0.180335  0.935548  -2.094773   33.020205
63593  QcMuo  0.635635 -0.473422   0.862670   64.578675
66136  74z30  1.084962 -0.312358   6.990771  127.643327
69163  AGS9W -0.933795  0.116194  -3.655896   1

Т.к. мы не знаем, какие признаки в повторяющихся id более достоверные и  мало дубликатов, то оставим датасеты без изменений.
Примем решение, что данные id не представляют "ценности", следовательно мы можем их удалить.

In [36]:
for i in df:
    i.drop(columns=['id'],
        axis=1,
        inplace=True)

In [37]:
# Проверка после удаления
for i in df:
    print(i.head())

         f0        f1        f2     product
0  0.705745 -0.497823  1.221170  105.280062
1  1.334711 -0.340164  4.365080   73.037750
2  1.022732  0.151990  1.419926   85.265647
3 -0.032172  0.139033  2.978566  168.620776
4  1.988431  0.155413  4.751769  154.036647
          f0         f1        f2     product
0 -15.001348  -8.276000 -0.005876    3.179103
1  14.272088  -3.475083  0.999183   26.953261
2   6.263187  -5.948386  5.001160  134.766305
3 -13.081196 -11.506057  4.999415  137.945408
4  12.702195  -8.147433  5.004363  134.766305
         f0        f1        f2     product
0 -1.146987  0.963328 -0.828965   27.758673
1  0.262778  0.269839 -2.530187   56.069697
2  0.194587  0.289035 -5.586433   62.871910
3  2.236060 -0.553760  0.930038  114.572842
4 -0.515993  1.716266  5.899011  149.600746


Т.к в проекте необходимо использовать линейную регрессию, то проверим корреляцию количественных признаков, чтобы не затруднить оценку и анализ общего результата, и исключить возникновение мультиколлинеарности.

In [38]:
for i in df:
    # Выделение количественных признаков ипостроение матриц корреляции
    print(i[['f0', 'f1', 'f2']].corr())
    print()

          f0        f1        f2
f0  1.000000 -0.440723 -0.003153
f1 -0.440723  1.000000  0.001724
f2 -0.003153  0.001724  1.000000

          f0        f1        f2
f0  1.000000  0.182287 -0.001777
f1  0.182287  1.000000 -0.002595
f2 -0.001777 -0.002595  1.000000

          f0        f1        f2
f0  1.000000  0.000528 -0.000448
f1  0.000528  1.000000  0.000779
f2 -0.000448  0.000779  1.000000



In [39]:
for i in df:
    # Выделение количественных признаков ипостроение матриц корреляции
    print(i.corr())
    print()

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000

               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000

               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000



По результатам провреки корреляции количественных признаков всех датасетов, самый большой коэффициет составил (по абсолютной величине) 0.4, следовательно использование таких признаков не должно привести к мулльтиколлениарности и по условию задачи все признаки значимы. Оставляем все признаки.

**Вывод:**
По результатам подготовки данных и их анализа не выявлено необходимости в дополнительной обаботке, но обнаружено, что при построении моделей необходимо масшатабировать признаки, чтобы избежать искажаения результаов.

## Обучение и проверка модели

### Разделение данных на обучающую и валидационную выборки в соотношении 75:25 и масштабирование признаков

In [40]:
# Разделение признаков и целевого признака по 1 региону
features_d0 = d0.drop(['product'], axis = 1)
target_d0 = d0['product']

In [41]:
# Разделение признаков и целевого признака по 2 региону
features_d1 = d1.drop(['product'], axis = 1)
target_d1 = d1['product']

In [42]:
# Разделение признаков и целевого признака по 3 региону
features_d2 = d2.drop(['product'], axis = 1)
target_d2 = d2['product']

In [43]:
#Выделим количественные признаки
numeric = ['f0', 'f1', 'f2']

In [44]:
# Разделение признаков на обучающую и валидационную выборки в соотношении 75:25 по 1 региону
features_train_d0, features_valid_d0, target_train_d0, target_valid_d0 = train_test_split(
    features_d0,
    target_d0,
    test_size = 0.25,
    random_state = 12345
)
# Игнорирование ошибки SettingWithCopy
pd.options.mode.chained_assignment = None

# Используем StandardScaler
scaler = StandardScaler()
scaler.fit(features_train_d0[numeric])

# Нормируем количественные признаки тренировочной и валидационной  выборки
features_train_d0[numeric] = scaler.transform(features_train_d0[numeric])
features_valid_d0[numeric] = scaler.transform(features_valid_d0[numeric])

# Проверим размерности выборок
print(features_train_d0.shape)
print(features_valid_d0.shape)
print(target_train_d0.shape)
print(target_valid_d0.shape)

#Проверка масштабирования
print(features_train_d0.head())
print(features_valid_d0.head())

(75000, 3)
(25000, 3)
(75000,)
(25000,)
             f0        f1        f2
27212 -0.544828  1.390264 -0.094959
7866   1.455912 -0.480422  1.209567
62041  0.260460  0.825069 -0.204865
70185 -1.837105  0.010321 -0.147634
82230 -1.299243  0.987558  1.273181
             f0        f1        f2
71751  0.517917 -0.610097 -0.126226
80493  0.568391 -0.086063 -0.814914
2655   0.805688 -1.613289 -1.341342
53233  0.222503 -1.355437 -0.597275
91141 -0.087941  1.433113 -0.544588


In [45]:
# Разделение признаков на обучающую и валидационную выборки в соотношении 75:25 по 2 региону
features_train_d1, features_valid_d1, target_train_d1, target_valid_d1 = train_test_split(
    features_d1,
    target_d1,
    test_size = 0.25,
    random_state = 12345
)
# Игнорирование ошибки SettingWithCopy
pd.options.mode.chained_assignment = None

# Используем StandardScaler
scaler = StandardScaler()
scaler.fit(features_train_d1[numeric])

# Нормируем количественные признаки тренировочной и валидационной  выборки
features_train_d1[numeric] = scaler.transform(features_train_d1[numeric])
features_valid_d1[numeric] = scaler.transform(features_valid_d1[numeric])

# Проверим размерности выборок
print(features_train_d1.shape)
print(features_valid_d1.shape)
print(target_train_d1.shape)
print(target_valid_d1.shape)

#Проверка масштабирования
print(features_train_d1.head())
print(features_valid_d1.head())

(75000, 3)
(25000, 3)
(75000,)
(25000,)
             f0        f1        f2
27212 -0.850855  0.624428  0.296943
7866   1.971935  1.832275  0.294333
62041  1.079305  0.170127 -0.296418
70185 -1.512028 -0.887837 -0.880471
82230 -1.804775 -0.718311 -0.293255
             f0        f1        f2
71751 -0.168616  0.571372  0.297471
80493  0.878384 -1.772903 -0.293901
2655  -0.852976 -0.004986 -0.876502
53233  1.496940 -1.146077 -0.294011
91141  0.552177  0.057961 -1.457764


In [46]:
# Разделение признаков на обучающую и валидационную выборки в соотношении 75:25 по 3 региону
features_train_d2, features_valid_d2, target_train_d2, target_valid_d2 = train_test_split(
    features_d2,
    target_d2,
    test_size = 0.25,
    random_state = 12345
)
# Игнорирование ошибки SettingWithCopy
pd.options.mode.chained_assignment = None

# Используем StandardScaler
scaler = StandardScaler()
scaler.fit(features_train_d2[numeric])

# Нормируем количественные признаки тренировочной и валидационной  выборки
features_train_d2[numeric] = scaler.transform(features_train_d2[numeric])
features_valid_d2[numeric] = scaler.transform(features_valid_d2[numeric])

# Проверим размерности выборок
print(features_train_d2.shape)
print(features_valid_d2.shape)
print(target_train_d2.shape)
print(target_valid_d2.shape)

#Проверка масштабирования
print(features_train_d2.head())
print(features_valid_d2.head())

(75000, 3)
(25000, 3)
(75000,)
(25000,)
             f0        f1        f2
27212 -0.526160  0.776329 -0.400793
7866  -0.889625 -0.404070 -1.222936
62041 -1.133984  0.208576  0.296765
70185  1.227045  1.570166 -0.764556
82230 -0.194289  0.878312  0.840821
             f0        f1        f2
71751 -0.836717 -2.227439 -0.078489
80493 -0.821648  0.740237 -1.000811
2655  -2.651336 -0.235721 -0.244546
53233  1.077998  0.938090  0.511313
91141 -1.173938  2.387270  1.034363


### Обучение модели и выполнение предсказания на валидационной выборке (RMSE, ср-й запас предсказанного сырья)

In [47]:
#Обучение и предсказание модели по 1-му месторождению
model_0 = LinearRegression()
# обучение модели на тренировочной выборке
model_0.fit(features_train_d0,target_train_d0) 
# предсказания модели на валидационной выборке
predictions_valid_0 = pd.Series(model_0.predict(features_valid_d0))
# ср-ee значениепредсказания модели на валидационной выборке
predictions_valid_0_mean = predictions_valid_0.mean()
# расчет значения RMSE
RMSE = mean_squared_error(target_valid_d0,predictions_valid_0)**0.5

print("Средний запас предсказанного сырья в месторождении 1:", predictions_valid_0_mean)
print("Средний фактический запас сырья в месторождении 1:", target_valid_d0.mean())
print("RMSE в месторождении 1:", RMSE)

Средний запас предсказанного сырья в месторождении 1: 92.59256778438035
Средний фактический запас сырья в месторождении 1: 92.07859674082927
RMSE в месторождении 1: 37.5794217150813


In [48]:
#Обучение и предсказание модели по 2-му месторождению
model_1 = LinearRegression()
# обучение модели на тренировочной выборке
model_1.fit(features_train_d1,target_train_d1) 
# предсказания модели на валидационной выборке
predictions_valid_1 = pd.Series(model_1.predict(features_valid_d1))
# ср-ee значениепредсказания модели на валидационной выборке
predictions_valid_1_mean = predictions_valid_1.mean()
# расчет значения RMSE
RMSE = mean_squared_error(target_valid_d1,predictions_valid_1)**0.5

print("Средний запас предсказанного сырья в месторождении 2:", predictions_valid_1_mean)
print("Средний фактический запас сырья в месторождении 2:", target_valid_d1.mean())
print("RMSE в месторождении 2:", RMSE)

Средний запас предсказанного сырья в месторождении 2: 68.728546895446
Средний фактический запас сырья в месторождении 2: 68.72313602435997
RMSE в месторождении 2: 0.893099286775617


In [49]:
#Обучение и предсказание модели по 3-му месторождению
model_2 = LinearRegression()
# обучение модели на тренировочной выборке
model_2.fit(features_train_d2,target_train_d2) 
# предсказания модели на валидационной выборке
predictions_valid_2 = pd.Series(model_2.predict(features_valid_d2))
# ср-ee значениепредсказания модели на валидационной выборке
predictions_valid_2_mean = predictions_valid_2.mean()
# расчет значения RMSE
RMSE = mean_squared_error(target_valid_d2,predictions_valid_2)**0.5

print("Средний запас предсказанного сырья в месторождении 2:", predictions_valid_2_mean)
print("Средний фактический запас сырья в месторождении 2:", target_valid_d2.mean())
print("RMSE в месторождении 3:", RMSE)

Средний запас предсказанного сырья в месторождении 2: 94.96504596800489
Средний фактический запас сырья в месторождении 2: 94.88423280885438
RMSE в месторождении 3: 40.02970873393434


### Вывод

Все модели предсказывают значения близкие к фактическим. Фактические средние запасы сырья отличаются от предсказанных для каждой модели в: 1 регионе - на сотые, во 2 регионе - на десятые и в 3 регирне - на сотые.

Лучшим прогназированием обладает модель 2, т.к.(RMSE - 0.89), далее 3 и 1 модели, но оно очень уступает по запасам сырья в отличие от остальных.

Самым большым запасом сырья по предсказаниям обладает 3 регион.

## Подготовка к расчёту прибыли

### Ключевые значения для расчётов

In [50]:
# кол-во точек для исследования
points_to_explore = 500
# кол-во точек для разработки
points_for_development = 200
# бюджет на разработку скважин в регионе,тысяч руб.
budget = 1*10**7
# доход с 1 барреля сырья, тысяч руб.
revenue = 450

### Рассчет достаточного объёма сырья для безубыточной разработки новой скважины и сравнение среднего объма запаса в скажине каждого региона

Чтобы найти достаточный объем сырья для безубыточной разработки новой скважины:
- определить допустимые средние затраты на разработку одной скважины - разделить бюджет на количество скважин;
- определить количество в тыс. баррель необходимо для покрытия затрат на 1 скважину.

In [51]:
mean_costs = budget / points_for_development
sufficient_product_volume = mean_costs / revenue
print('Достаточный объем сырья для безубыточной разработки 1 скважины:', round(sufficient_product_volume, 2), 'тысяч баррелей')

Достаточный объем сырья для безубыточной разработки 1 скважины: 111.11 тысяч баррелей


Расчет среднего объма запаса в скажине каждого региона

In [52]:
print('Средние запасы в 1 регионе:', round(d0['product'].mean(),1), 'тысяч баррелей')
print('Средние запасы во 2 регионе:', round(d1['product'].mean(),1), 'тысяч баррелей')
print('Средние запасы в 3 регионе::', round(d2['product'].mean(),1), 'тысяч баррелей')

Средние запасы в 1 регионе: 92.5 тысяч баррелей
Средние запасы во 2 регионе: 68.8 тысяч баррелей
Средние запасы в 3 регионе:: 95.0 тысяч баррелей


**Вывод:**
- Достаточный объем сырья для безубыточной разработки 1 скважины составил  111.11 тысяч баррелей. 
- Средние значения объма запасов сырья в скважине для всех регионов ниже достаточного объема, что говорит о риске неокупаемости при разработке. 

## Расчёт прибыли и рисков 

### Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- Определяем скважины с максимальными значениями предсказаний (200 скавжин). 
- Суммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
- Производим расчет прибыли для полученного объёма сырья.

In [53]:
def profit(target_valid, predictions_valid, points_for_development):
    # выбор 200 скважин с максимальными значениями предсказаний объемом нефти
    a = predictions_valid.sort_values(ascending=False)
    # Суммируем целевое значение объёма сырья, соответствующее этим предсказаниям
    b = target_valid.reset_index(drop=True)[a.index][:points_for_development].sum() 
    # прибыль в тысячах рублей с 200 скважин
    с = b * revenue -  budget
    return с

In [54]:
print('Прибыль с 200 скважин с максимальными значениями предсказаний объемом нефти по региону:')
print('Регион №1:', round(profit(target_valid_d0, predictions_valid_0, points_for_development),2), 'тыс. руб.')
print('Регион №2:', round(profit(target_valid_d1, predictions_valid_1, points_for_development),2), 'тыс. руб.')
print('Регион №3:', round(profit(target_valid_d2, predictions_valid_2, points_for_development),2), 'тыс. руб.')

Прибыль с 200 скважин с максимальными значениями предсказаний объемом нефти по региону:
Регион №1: 3320826.04 тыс. руб.
Регион №2: 2415086.7 тыс. руб.
Регион №3: 2710349.96 тыс. руб.


В результате расчета получены достаточно высокие значения прибыли для каждого региона от 2.4 до 3.3 млрд. руб., где регион №1 самый прибыльный.

### Посчитайте риски и прибыль для каждого региона

Создадим новую функцию для расчета прибыли, т.к. не удалось отладить функцию (п.4.1) с применением техники Bootstrap с 1000 выборок

In [55]:
def profit_new(target_valid, predictions_valid, points_for_development):
    # выбор 200 скважин с максимальными значениями предсказаний объемом нефти
    a = predictions_valid.sort_values(ascending=False)
    # Суммируем целевое значение объёма сырья, соответствующее этим предсказаниям
    b = target_valid[a.index][:points_for_development]
    # прибыль в тысячах рублей с 200 скважин
    с = b.sum() * revenue - budget
    return с

- Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли;
- Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков (отрицательная прибыль).

In [56]:
def bootstrap (target_valid, predictions_valid):
    state = np.random.RandomState(12345)
    # значения прибыли для 1000 выборок
    values = []
    
    for i in range(1000):
         # для target_valid сбрасываем индексы и случайно выбираем нужное количество скважин
        target_subsample = target_valid.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        # выбираем строки соответствующие отобранным строкам в целевом признаке
        preds_subsample = predictions_valid[target_subsample.index]
        # добавляем прибыль в список прибылей всех выборок
        values.append(profit_new(target_subsample, preds_subsample, 200))
      
    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    
    print('Вероятность убытков:',st.percentileofscore(values, 0),'%')
    print('Среднее значение bootstrap:', round(values.mean(),2), 'тысяч руб.')
    print('Верхняя граница 95%-го доверительного интервала:', round(higher,2))
    print('Нижняя граница 95%-го доверительного интервала:', round(lower,2))
    print()
print("Регион №1")
bootstrap(target_valid_d0, predictions_valid_0)

print("Регион №2")
bootstrap(target_valid_d1, predictions_valid_1)

print("Регион №3")
bootstrap(target_valid_d2, predictions_valid_2)

Регион №1
Вероятность убытков: 6.0 %
Среднее значение bootstrap: 425938.53 тысяч руб.
Верхняя граница 95%-го доверительного интервала: 947976.35
Нижняя граница 95%-го доверительного интервала: -102090.09

Регион №2
Вероятность убытков: 1.0 %
Среднее значение bootstrap: 515222.77 тысяч руб.
Верхняя граница 95%-го доверительного интервала: 931547.59
Нижняя граница 95%-го доверительного интервала: 68873.23

Регион №3
Вероятность убытков: 6.4 %
Среднее значение bootstrap: 435008.36 тысяч руб.
Верхняя граница 95%-го доверительного интервала: 969706.95
Нижняя граница 95%-го доверительного интервала: -128880.55



**Вывод:**
Анализируя  полученные результаты можно сделать вывод о том, что:

- При расчете средней прибыли все регионы являются прибыльными, где:

1) Прибыль в регионе №1 составила - 425938.53 тысяч руб.;

2) Прибыль в регионе №2 составила - 515222.77 тысяч руб.;

3) Прибыль в регионе №3 составила - 435008.36 тысяч руб.;

- Рассматривая вероятность убытков в регионах №1 и №3 значения составляют 6% и :6.4%, что не соответствует требованиям зааказчика, т.е. превышают установленный порог в 2.5%. Также стоит отметить, что нижние границы 95%-го доверительного интервала в данных регионах являются отрицательным т.е. имеют отрицательную прибыль.

- Рассматривая вероятность убытков в регионе №2, где значение составляет 2%, что удовлетворяет условия заказчика, а также положительные значения границ 95%-го доверительного интервала. Можно сделать вывод, что по требованиям заказчика соответствует регион №2, т.е. его можно рекомендовать для разработки месторождения.